### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

# Read in the datasets
movies = pd.read_csv('./data/original_movies.dat',
                     delimiter='::',
                     header=None,
                     names=['movie_id', 'movie', 'genre'],
                     dtype={'movie_id': object}, engine='python')

reviews = pd.read_csv('./data/original_ratings.dat',
                      delimiter='::',
                      header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object},
                      engine='python')

# Reduce the size reviews dataset
reviews = reviews.loc[:100000,:]

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [18]:
reviews.drop_duplicates().shape[0]

100001

In [16]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [24]:
movies['genre'].drop_duplicates().dropna().shape[0]

2736

In [19]:
# Use this cell to find the following information:
# number of movies
n_movies = movies['movie_id'].unique().shape[0]
# number of ratings
n_ratings = reviews.drop_duplicates().shape[0]
# number of different genres
n_genres = movies['genre'].drop_duplicates().dropna().shape[0]
# number of unique users
n_users = reviews['user_id'].unique().shape[0]
# number of missing ratings
n_missing_rating = reviews['rating'].isna().sum()
# the `average`, `min`, and `max` ratings given
rating_avg = reviews['rating'].mean()
rating_min = reviews['rating'].min()
rating_max = reviews['rating'].max()

In [36]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 8022
b = 10
c = 7
d = 35479
e = 15
f = 0
g = 4
h = 100001
i = 28

# Guide: replace "ENTER YOUR ANSWER HERE" with the matching letter above.
# For example, 'The number of movies in the dataset': d,

dict_sol1 = {
'The number of movies in the dataset': d, 
'The number of ratings in the dataset': h,
'The number of different genres': i, 
'The number of unique users in the dataset': a, 
'The number missing ratings in the reviews dataset': f, 
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': f,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's for each genre

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [51]:
movies.head()

,movie_id,movie,genre,year,1800s,1900s,2000s
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [43]:
movies['movie'].apply(lambda s: s.split('(')[1][:-1]).isna().sum()

0

In [70]:
movies['genre'].explode().dropna().unique()

array(['Documentary', 'Short', 'Horror', 'Comedy', 'Action', 'Adventure',
       'Fantasy', 'Sci-Fi', 'Crime', 'Western', 'Drama', 'Romance',
       'History', 'Family', 'War', 'Sport', 'Biography', 'Mystery',
       'Thriller', 'Animation', 'Music', 'Musical', 'Film-Noir', 'Adult',
       'Talk-Show', 'News', 'Reality-TV', 'Game-Show'], dtype=object)

In [50]:
# Movies
movies['year']  = movies['movie'].apply(lambda s: int(s.split('(')[1][:-1]))
movies['1800s'] = movies['year'].apply(lambda x: int((x>=1800)&(x<1900)))
movies['1900s'] = movies['year'].apply(lambda x: int((x>=1900)&(x<2000)))
movies['2000s'] = movies['year'].apply(lambda x: int((x>=2000)))

In [71]:
movies['genre'] = movies['genre'].str.split('|')

genres = movies['genre'].explode().dropna().unique()

for genre in genres:
    movies[genre] = movies['genre'].apply(lambda x: int(np.isin(x, genre).sum()>0))
    

c:\Users\Pontello\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
c:\Users\Pontello\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
c:\Users\Pontello\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
c:\Users\Pontello\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
c:\Users\Pontello\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise compa

In [73]:
movies.head()

,movie_id,movie,genre,year,1800s,1900s,2000s,Documentary,Short,Horror,...,Thriller,Animation,Music,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),"[Documentary, Short]",1894,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),"[Documentary, Short]",1895,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),"[Documentary, Short]",1896,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),"[Short, Horror]",1896,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [54]:
movies['genre'] = movies['genre'].str.split('|')

In [47]:
movies['year'].apply(lambda x: (x>=1800)|(x<1900))

0        True
1        True
2        True
3        True
4        True
         ... 
35474    True
35475    True
35476    True
35477    True
35478    True
Name: year, Length: 35479, dtype: bool

In [ ]:
# Click `+` button on the nav menu to add more cells for work

In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()